# Threads
* Reading: OS essentials Page 163-190
* Abstraction of a processor
* a *thread of control*
### Why threads
* A natural means for dealing with concurrency:
  * multiprocessor and equally useful on uniprocesors
### Processes vs. Threads
* Both provide concurrency
* What the difference?
  * Two single-thread processes vs. one two-threaded process
    * First, for an existing process, it is much cheaper to add a thread than creating a new process
    * Context switching: it is also cheaper sharing context between two threads in the same process

# Matrix Multiplication
* C=A*B
* A: m x n
  * 2 x 5
* B: n x p
  * 5 x 3
* C: ?
  * 2 x 3
  * m x p


What the heck

```c
#include <stdio.h>
#include <pthread.h>
#include<string.h>

#define M 3
#define N 4
#define P 5

long A [M][N]
long B [N][P]
long C [M][P]


long A[m]n], B[n][p], C[m][p];
for (i=0; i < m; i ++) // create worker threads
{
    pthread_create(&thr[i],0, matmult, i);
    // &thr[i] is the id of the thread 
    // 0 is the 
    // matmult is the task, 
    // i is the parameter you need to provice for the function
}

```

# Example Question
```c
#include "csapp.h"
#define N 2
void *thread(void *vargp);

char **ptr; /* Global variable */ // line:conc:sharing:ptrdec

int main()
{
    int i; // stored in the stack for the main thread
    pthread_t tid; // stored in the stack for the main thread
    char *msgs[N] = {
        "Hello from foo",
        "Hello from bar"};
    ptr = msgs;                 // since ptr is a global variable, every peer thread can access whatever it points to
                                // so now everything can access msgs
    for (i = 0; i < N; i++)
    {
        Pthread_create(&tid, NULL, thread, (void *)i);
    }
    Pthread_exit(NULL); // waits for all other threads to finsih
}

void *thread(void *vargp)
{
    int myid = (int)vargp;
    static inc cnt = 0;                                    // line:conc:sharing:cntdec
    // so this is static, so it is shared
    printf("[%d]: %s (cnt=%d)\n", myid, ptr[myid], ++cnt); // line:conc:sharing:stacj
    return NULL;
}
```

## What is accessed
| Variable Instance | Main Thread | Peer Thread 0 | Peer Thread 1 |
| ----------------- | ----------- | ------------- | ------------- |
| ptr               | yes         | yes           | yes           |
| cnt               | no          | yes           | yes           |
| msg.m             | yes         | yes           | yes           |
| i.m               | yes         | no            | no            |
| myid.p0           | no          | yes           | no            |
| myid.p1           | no          | no            | yes           |

## Thread
Has its own:
* Thread ID
* Stack
* Stack Pointer ($rsp)
* Program Counter ($rip)
* FLAG (Condition Codes)
* General Purpose Registers

Sharing:
* Virtual Address Space
* Code
* Data
* Heap
* Open Files

```c
pthread_create([], 0 task, arg); 
pthread_join(); // there is some information mantained, like how it is terminated
// similar to wait()
pthread_exit(); // waits for all other peers to finish

wait(); // this must be called if you want the child to do something, and THEN the parent does something.
// it "waits" until the child is done
// wait vs waitpid, you use waitpid if there are multiple children (you specify which pip/child to wait for)
// also use waitpid if the parent doesnt terminate. this is becasue there wont be any chance for the init process to cleanup since it never terminates
// ^ like a shell or server
```

### Contex
* When you switch between threads, there are a few things of info you need to keep and tell the next thread.
  * Thread ID
  * Stack
  * Stack Pointer
  * Program Counter
  * Condition codes
  * General-purpose Register Values

### Data Sharing
* Each thread shares the rest of the process context with other threads:
  * The entire user virtual address space
  * Read-only text (code)
  * Read/Write data
  * Heap
  * Same set of open files
  * Library Code

### Strack SIze (2MB by default)
```c
pthread_t thread;
pthread_attr_t thr_attr;
pthread_attr_init(&thr_attr);
pthread_attr_setstacjsize(&thr_attr,20*1024*1024); // this is 20MB
...
pthread_create(&thread, &thr_attr, startroutine, arg);
...
```

### Mutexes
* A synchronization construct providing mutual exclusion
* Code locking:
  * only one thread is executoing a particular piece of code at once
* Data locking
  * only one htread is accessing a particular thread at once
* A mutexe must be initialized

```c
pthread_mutex_t m = PTHREAD_MUTEX_INITIALIZER;
int x;
Pthread_mutex_lock(&m);
x = x+1;
```
* An important restriction:
  * the thread that locked a mutex should be the thread that unlocks it.

```c
pthread_mutex_lock(&m);
// critical selection
pthread_mutex_unlock(&m)
```

### Singly Linked List
```c
typedef struct node{
    pthread_mutex_t mutex;
    int value;
    struct node *next;
} node_t;

void add_after (node_t *after, node_t *new){
    pthread_mutex_lock(&after->mutex);
    new
    rest on powerpoint...
    


```

### Doubly Linked List
```c
// apparently these functions are bad and have bad side effects...
void add_after(node_t *after, node_t *new){
    pthread_mutex_lock(&after->mutex);
    after->next->prev = new;
    new->next = after->next;
    new->prev = after;
    after->next = new;
    pthread_mutex_unlock(&after->mutex);
}
void delete(npde_t *old){
    pthread_mutex_lock(&old->mutex);
    old->prev->next = old->next;
    old->next->prev = old->prev;
    pthread_mutex_unlock(&old->mutex);
}



`.